In [1]:
import joblib
import pandas as pd

In [2]:
sdf = pd.read_csv('dataset/sdf_dataset.csv')

In [3]:
# Load the CountVectorizer object
count = joblib.load('model/count_vectorizer.pkl')

# Load the cosine similarity matrix
cosine_sim = joblib.load('model/cosine_sim_matrix.pkl')

# Load the trained SVD model
model = joblib.load('model/svd_recommender_model.pkl')

# Load the id_map and indices_map
id_map = joblib.load('model/id_map.pkl')
indices_map = joblib.load('model/indices_map.pkl')
indices = joblib.load('model/indices_series.pkl')

In [4]:
def hybrid_recommendations(userId, title):
    title = str(title).title()
    idx = indices[title]
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = sdf.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: model.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [5]:
hybrid_recommendations(6, 'toy story')

,title,vote_count,vote_average,year,id,est
3817,"Monsters, Inc.",6150.0,7.5,2001,585,4.267420
2514,Toy Story 2,3914.0,7.3,1999,863,3.919054
8555,The Lego Movie,3127.0,7.5,2014,137106,3.828403
7589,Toy Story 3,4710.0,7.6,2010,10193,3.822331
7133,A Matter of Loaf and Death,120.0,7.2,2008,14447,3.693756
1875,A Bug's Life,2379.0,6.8,1998,9487,3.600464
4317,"The Looney, Looney, Looney Bugs Bunny Movie",13.0,6.6,1981,41394,3.325841
7218,Kung Fu Panda: Secrets of the Furious Five,80.0,6.5,2008,15854,3.310732
1824,Antz,1320.0,6.0,1998,8916,3.193978
6354,Luxo Jr.,148.0,7.1,1986,13925,3.134774


In [6]:
import streamlit as st
import pickle

# Load your pre-trained models and any other necessary objects
def load_model():
    with open("", "rb") as file:
        model = pickle.load(file)
    return model

# Title of the App
st.title("Recommender System")

# Input: User could enter an item or any query relevant to your system
user_input = st.text_input("Enter your preferences or the item name:")

# Button to generate recommendations
if st.button("Recommend"):
    # Fetch the recommendation based on user input
    model = load_model()
    recommendations = model.recommend(user_input)  # Replace with your recommendation logic

    # Display recommendations
    st.write("Top recommendations for you:")
    for item in recommendations:
        st.write(item)


2024-10-15 10:59:49.463 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 10:59:50.779 
  command:

    streamlit run C:\Users\medoo\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-10-15 10:59:50.781 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 10:59:50.782 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 10:59:50.783 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 10:59:50.785 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 10:59:50.786 Session state does not function when running a script without `streamlit run`
2024-10-15 10:59:50.787 Thread 'MainThread': missing 

In [12]:
import streamlit as st
import joblib
import pandas as pd

# Function to load the content-based model components
def load_content_based_model():
    # Load the CountVectorizer object
    count_vectorizer = joblib.load('model/count_vectorizer.pkl')

    # Load the cosine similarity matrix
    cosine_sim = joblib.load('model/cosine_sim_matrix.pkl')
    
    return count_vectorizer, cosine_sim

# Function to load the collaborative filtering model (SVD)
def load_collaborative_model():
    # Load Surprise SVD model
    svd_model = joblib.load('model/svd_recommender_model.pkl')  # No unpacking
    return svd_model

# Hybrid recommendation logic (content-based and collaborative filtering)
def hybrid_recommendations(userId, title, cosine_sim, sdf, indices, indices_map, svd_model):
    # Ensure the title is correctly capitalized
    title = str(title).title()
    
    # Get the index of the title from the indices
    idx = indices[title]
    
    # Get pairwise similarity scores of all movies with the chosen movie
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    
    # Sort the movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top 25 most similar movies
    sim_scores = sim_scores[1:26]
    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    # Create a DataFrame for the 25 most similar movies
    movies = sdf.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    
    # Apply collaborative filtering to re-rank the movies
    movies['est'] = movies['id'].apply(lambda x: svd_model.predict(userId, indices_map.loc[x]['movieId']).est)
    
    # Sort the movies based on the estimated rating from the SVD model
    movies = movies.sort_values('est', ascending=False)
    
    # Return the top 10 recommended movies
    return movies.head(10)

# Streamlit App
st.title("Hybrid Recommender System")

# Load models
count_vectorizer, cosine_sim = load_content_based_model()
svd_model = load_collaborative_model()

# Load the dataset (sdf) and indices for movie titles
sdf = pd.read_csv('dataset/sdf_dataset.csv')

indices_map = joblib.load('model/indices_map.pkl')
indices = joblib.load('model/indices_series.pkl')

# User input: Enter preferences or movie title and userId
user_id_input = st.number_input("Enter your user ID:", min_value=1, step=1)
movie_input = st.text_input("Enter the movie title:")

# Button to generate recommendations
if st.button("Recommend"):
    if movie_input:
        # Generate hybrid recommendations using the provided user ID and movie title
        recommendations = hybrid_recommendations(user_id_input, movie_input, cosine_sim, sdf, indices, indices_map, svd_model)
        
        # Display recommendations
        st.write("Top recommendations for you:")
        st.write(recommendations[['title', 'year', 'vote_average', 'est']])
    else:
        st.write("Please enter a movie title.")


2024-10-15 11:39:35.256 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 11:39:35.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 11:39:38.093 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 11:39:38.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 11:39:38.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 11:39:38.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 11:39:38.098 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 11:39:38.099 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar